In [14]:
%config IPCompleter.greedy=True
%matplotlib inline

In [15]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats

In [16]:
# Import the requests library.
import requests

# Import the API key.
from config import weather_api_key

In [17]:
# Import the datetime module from the datetime library.
from datetime import datetime

In [18]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key
print(url)

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=c86c27774e3709fc1f81f95454134804


In [19]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)
lat_lngs

In [20]:
# Use the citipy module to determine city based on latitude and longitude.
from citipy import citipy

In [21]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

751

In [22]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

Beginning Data Retrieval     
-----------------------------


In [23]:
# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        weather_description = city_weather["weather"][0]["description"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Current Description": weather_description,
                          "Country": city_country,
                          "Date": city_date})

    # If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

    # Indicate that Data Loading is complete.
    print("-----------------------------")
    print("Data Retrieval Complete      ")
    print("-----------------------------")

ta Retrieval Complete      
-----------------------------
Processing Record 2 of Set 13 | bubaque
-----------------------------
Data Retrieval Complete      
-----------------------------
Processing Record 3 of Set 13 | dubbo
-----------------------------
Data Retrieval Complete      
-----------------------------
Processing Record 4 of Set 13 | itigi
-----------------------------
Data Retrieval Complete      
-----------------------------
Processing Record 5 of Set 13 | puerto carreno
-----------------------------
Data Retrieval Complete      
-----------------------------
Processing Record 6 of Set 13 | foam lake
-----------------------------
Data Retrieval Complete      
-----------------------------
Processing Record 7 of Set 13 | paragominas
-----------------------------
Data Retrieval Complete      
-----------------------------
Processing Record 8 of Set 13 | lolua
City not found. Skipping...
-----------------------------
Data Retrieval Complete      
---------------------------

In [24]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Country,Date
0,Albany,42.60,-73.97,60.01,96,0,6.53,clear sky,US,2020-08-31 01:17:07
1,Mataura,-46.19,168.86,61.88,46,0,15.19,clear sky,NZ,2020-08-31 01:17:07
2,Punta Arenas,-53.15,-70.92,32.00,90,12,13.87,few clouds,CL,2020-08-31 01:17:07
3,Tuktoyaktuk,69.45,-133.04,46.40,81,90,16.11,overcast clouds,CA,2020-08-31 01:17:08
4,Saint-Philippe,-21.36,55.77,66.20,68,20,3.36,few clouds,RE,2020-08-31 01:17:08
5,Arraial Do Cabo,-22.97,-42.03,75.20,78,0,12.75,clear sky,BR,2020-08-31 01:17:08
6,Pietarsaari,63.65,22.68,57.00,93,100,2.24,light rain,FI,2020-08-31 01:17:09
7,Victoria,22.29,114.16,89.01,62,53,5.99,light rain,HK,2020-08-31 01:17:09
8,Derzhavinsk,51.10,66.32,66.72,46,0,6.96,clear sky,KZ,2020-08-31 01:13:29
9,Upernavik,72.79,-56.15,39.87,88,100,10.22,overcast clouds,GL,2020-08-31 01:17:09


In [25]:
# Create the output file (CSV).
output_data_file = "weather_data/weatherpy_challenge.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")